In [1]:
from pathlib import Path
import os

import pandas as pd
import seaborn as sns

import src
from src.sim.agent import Agent
from src.sim.sim import Sim


def get_population_df(state, n_agents, n_internal_runs, save=True):
    
    hh_counter = 0

    dfs = []
    for rep in range(n_internal_runs):
        agent_data = []

        print(round((rep/n_internal_runs)*100),"%")
        
        model = Sim(state)

        population = model.create_soep_population(N=n_agents, agent_class=Agent)

        for i, hh in enumerate(population):
            hh_size = len(hh)
            
            hh_id = hh_counter
            hh_counter += 1
            
            for agent in hh:
                agent.hh_id = hh_id
                agent.hh_size = hh_size
                agent.rep = rep
                agent_data.append(vars(agent))
        
        df = pd.DataFrame(agent_data)
        
        dfs.append(df)
    
    df_total = pd.concat(dfs)
    df_total = df_total.reset_index(drop=True)
    df_total["state"] = state

    if save:
        population_data_folder = Path.joinpath(src.PATH, "important_outputs", "population")
        if not os.path.exists(population_data_folder):
            os.mkdir(population_data_folder)
        
        df_total.to_csv(
            Path.joinpath(population_data_folder, "population_" + str(state) + "_" + str(n_agents) + "_" + str(n_internal_runs) + ".csv"), 
            index=False,
            )

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#get_population_df(state=2, n_agents=100000, n_internal_runs=60)
#get_population_df(state=8, n_agents=100000, n_internal_runs=60)
#get_population_df(state=9, n_agents=100000, n_internal_runs=60)
#get_population_df(state=10, n_agents=100000, n_internal_runs=60)

In [3]:
dfs = [pd.read_csv(Path.joinpath(src.PATH, "important_outputs", "population", f"population_{state}_100000_60.csv")) for state in ("2", "8", "9", "10")]
df = pd.concat(dfs)
del(dfs)

# df = pd.read_csv(Path.joinpath(src.PATH, "minimal_data_set", "agent_population.csv") )

In [4]:
df.columns

Index(['name', 'residence_cell', 'x_grid_pos', 'y_grid_pos', 'population',
       'gender', 'age', 'infection', 'tick_of_exposure', 'tick_of_recovery',
       'tick_of_symptom_onset', 'home_cell', 'work_place', 'school',
       'kindergarten', 'fav_supermarkets', 'cell_of_infection', 'target_cell',
       'activity', 'activity_len_in_ticks', 'ticks_doing_this_activity',
       'group_dict', 'activities_done_today', 'stay_at_home', 'quarantine',
       'household_members', 'duration_s', 'duration_i', 'duration_r_a',
       'duration_r_m', 'nace2', 'nace2_short', 'work_hours_day_in_ticks',
       'hours_at_supermarket_in_ticks', 'student', 'hid', 'pid',
       'federal_state', 'hh_id', 'hh_size', 'rep', 'state'],
      dtype='object')

In [5]:
df.head()

,name,residence_cell,x_grid_pos,y_grid_pos,population,gender,age,infection,tick_of_exposure,tick_of_recovery,...,work_hours_day_in_ticks,hours_at_supermarket_in_ticks,student,hid,pid,federal_state,hh_id,hh_size,rep,state
0,2022343066048,NaN,NaN,NaN,<class 'list'>,2,43,s,NaN,NaN,...,7.0,0.952701,0,260916,2609102,2,0,4,0,2
1,2022343066192,NaN,NaN,NaN,<class 'list'>,2,4,s,NaN,NaN,...,0.0,0.952701,0,260916,2609104,2,0,4,0,2
2,2022343066240,NaN,NaN,NaN,<class 'list'>,2,7,s,NaN,NaN,...,0.0,0.952701,0,260916,2609103,2,0,4,0,2
3,2022343065904,NaN,NaN,NaN,<class 'list'>,1,42,s,NaN,NaN,...,7.0,1.000000,0,260916,2609101,2,0,4,0,2
4,2022343065808,NaN,NaN,NaN,<class 'list'>,2,76,s,NaN,NaN,...,0.0,2.000000,0,3100617,31006101,2,1,1,0,2


In [6]:
df["kindergarten"] = df["age"].apply(lambda age: (1 if age <= 5 else 0))
df["school"] = df["age"].apply(lambda age: (1 if 5 < age < 20  else 0))

df["working"] = 0
df.loc[(df["student"] == 0) & (df["work_hours_day_in_ticks"] > 0), "working"] = 1

df["none"] = 0
df.loc[(df["kindergarten"] == 0) & (df["school"] == 0) & (df["student"] == 0) & (df["working"] == 0), "none"] = 1


df["job_status"] = "None/other"
df.loc[(df["kindergarten"] == 1), "job_status"] = "Kindergarten kid"
df.loc[(df["school"] == 1), "job_status"] = "Pupil"
df.loc[(df["student"] == 1), "job_status"] = "Student"
df.loc[(df["job_status"] == "None/other") & (df["work_hours_day_in_ticks"] > 0), "job_status"] = "Working"

## Share of status

In [7]:
table = df.groupby("state")["job_status"].value_counts() / df.groupby("state")["job_status"].count()
table = table.round(3)
table

state  job_status      
2      None/other          0.432
       Working             0.340
       Pupil               0.143
       Kindergarten kid    0.051
       Student             0.035
8      Working             0.435
       None/other          0.361
       Pupil               0.131
       Kindergarten kid    0.050
       Student             0.023
9      Working             0.446
       None/other          0.360
       Pupil               0.125
       Kindergarten kid    0.048
       Student             0.021
10     Working             0.437
       None/other          0.377
       Pupil               0.126
       Kindergarten kid    0.045
       Student             0.014
Name: job_status, dtype: float64

In [8]:
print(table.to_latex())

\begin{tabular}{llr}
\toprule
   &         &  job\_status \\
state & job\_status &             \\
\midrule
2  & None/other &       0.432 \\
   & Working &       0.340 \\
   & Pupil &       0.143 \\
   & Kindergarten kid &       0.051 \\
   & Student &       0.035 \\
8  & Working &       0.435 \\
   & None/other &       0.361 \\
   & Pupil &       0.131 \\
   & Kindergarten kid &       0.050 \\
   & Student &       0.023 \\
9  & Working &       0.446 \\
   & None/other &       0.360 \\
   & Pupil &       0.125 \\
   & Kindergarten kid &       0.048 \\
   & Student &       0.021 \\
10 & Working &       0.437 \\
   & None/other &       0.377 \\
   & Pupil &       0.126 \\
   & Kindergarten kid &       0.045 \\
   & Student &       0.014 \\
\bottomrule
\end{tabular}



## Average houshold size by status

In [9]:
table = df.groupby(["state", "job_status"])["hh_size"].mean()
table

state  job_status      
2      Kindergarten kid    3.999386
       None/other          2.398013
       Pupil               4.157018
       Student             2.188056
       Working             2.428195
8      Kindergarten kid    4.169087
       None/other          2.207290
       Pupil               4.282583
       Student             2.779872
       Working             2.668880
9      Kindergarten kid    3.667491
       None/other          2.142372
       Pupil               3.971149
       Student             2.547139
       Working             2.490422
10     Kindergarten kid    3.511309
       None/other          2.285616
       Pupil               3.910949
       Student             2.132892
       Working             2.219677
Name: hh_size, dtype: float64

In [10]:
print(table.to_latex())

\begin{tabular}{llr}
\toprule
   &         &   hh\_size \\
state & job\_status &           \\
\midrule
2  & Kindergarten kid &  3.999386 \\
   & None/other &  2.398013 \\
   & Pupil &  4.157018 \\
   & Student &  2.188056 \\
   & Working &  2.428195 \\
8  & Kindergarten kid &  4.169087 \\
   & None/other &  2.207290 \\
   & Pupil &  4.282583 \\
   & Student &  2.779872 \\
   & Working &  2.668880 \\
9  & Kindergarten kid &  3.667491 \\
   & None/other &  2.142372 \\
   & Pupil &  3.971149 \\
   & Student &  2.547139 \\
   & Working &  2.490422 \\
10 & Kindergarten kid &  3.511309 \\
   & None/other &  2.285616 \\
   & Pupil &  3.910949 \\
   & Student &  2.132892 \\
   & Working &  2.219677 \\
\bottomrule
\end{tabular}



## Household members by status

In [11]:
# sum up statuses by households and save it for each agent
df["hh_n_kindergarten"] = df.groupby(["state", "hh_id"])["kindergarten"].transform(sum)
df["hh_n_school"] = df.groupby(["state", "hh_id"])["school"].transform(sum)
df["hh_n_uni"] = df.groupby(["state", "hh_id"])["student"].transform(sum)
df["hh_n_working"] = df.groupby(["state", "hh_id"])["working"].transform(sum)
df["hh_n_none"] = df.groupby(["state", "hh_id"])["none"].transform(sum)


df["hh_n_total"] = df["hh_n_kindergarten"] + df["hh_n_school"] + df["hh_n_uni"] + df["hh_n_working"] + df["hh_n_none"]

# calculate average number of statuses in household by status
df_household_state = df.groupby(["state", "job_status"])[["hh_n_kindergarten", "hh_n_school", "hh_n_uni", "hh_n_working", "hh_n_none", "hh_n_total"]].mean()
df_household_state = df_household_state.round(4)
df_household_state

hh_n_kindergarten  hh_n_school  hh_n_uni  \
state job_status                                                   
2     Kindergarten kid             1.3499       0.5905    0.0838   
      None/other                   0.0795       0.3465    0.0209   
      Pupil                        0.2095       1.9449    0.0312   
      Student                      0.1230       0.1620    1.0000   
      Working                      0.1937       0.4059    0.0686   
8     Kindergarten kid             1.5276       0.6327    0.0261   
      None/other                   0.0815       0.1961    0.0190   
      Pupil                        0.2432       2.0126    0.0303   
      Student                      0.0565       0.2888    1.1906   
      Working                      0.1620       0.4503    0.0569   
9     Kindergarten kid             1.4019       0.3561    0.0206   
      None/other                   0.0813       0.1684    0.0242   
      Pupil                        0.1370       1.8404    0.0347   
      Student                      0.0466       0.2598    1.1157   
      Working                      0.1379       0.4183    0.0366   
10    Kindergarten kid             1.0730       0.8116    0.0032   
      None/other                   0.0809       0.2303    0.0079   
      Pupil                        0.2913       1.7188    0.0310   
      Student                      0.0099       0.2980    1.1684   
      Working                      0.0986       0.3444    0.0158   

                        hh_n_working  hh_n_none  hh_n_total  
state job_status                                             
2     Kindergarten kid        1.2986     0.6766      3.9994  
      None/other              0.2147     1.7415      2.4033  
      Pupil                   0.9653     1.0299      4.1808  
      Student                 0.6746     0.2612      2.2208  
      Working                 1.4876     0.2725      2.4282  
8     Kindergarten kid        1.3984     0.5842      4.1691  
      None/other              0.3548     1.5585      2.2098  
      Pupil                   1.4664     0.5333      4.2859  
      Student                 1.0656     0.2959      2.8974  
      Working                 1.7132     0.2948      2.6772  
9     Kindergarten kid        1.2798     0.6091      3.6675  
      None/other              0.3821     1.4870      2.1430  
      Pupil                   1.4816     0.4840      3.9778  
      Student                 0.7691     0.4115      2.6027  
      Working                 1.5926     0.3086      2.4939  
10    Kindergarten kid        0.9498     0.6738      3.5113  
      None/other              0.4121     1.5555      2.2866  
      Pupil                   1.1880     0.6849      3.9140  
      Student                 0.4783     0.2052      2.1598  
      Working                 1.4057     0.3560      2.2206

In [15]:
df_hh = df[df["state"] == 2]
df_hh.head()

,name,residence_cell,x_grid_pos,y_grid_pos,population,gender,age,infection,tick_of_exposure,tick_of_recovery,...,state,working,none,job_status,hh_n_kindergarten,hh_n_school,hh_n_uni,hh_n_working,hh_n_none,hh_n_total
0,2022343066048,NaN,NaN,NaN,<class 'list'>,2,43,s,NaN,NaN,...,2,1,0,Working,1,1,0,2,0,4
1,2022343066192,NaN,NaN,NaN,<class 'list'>,2,4,s,NaN,NaN,...,2,0,0,Kindergarten kid,1,1,0,2,0,4
2,2022343066240,NaN,NaN,NaN,<class 'list'>,2,7,s,NaN,NaN,...,2,0,0,Pupil,1,1,0,2,0,4
3,2022343065904,NaN,NaN,NaN,<class 'list'>,1,42,s,NaN,NaN,...,2,1,0,Working,1,1,0,2,0,4
4,2022343065808,NaN,NaN,NaN,<class 'list'>,2,76,s,NaN,NaN,...,2,0,1,None/other,0,0,0,0,1,1


In [21]:
df_hh[df_hh["job_status"] == "Student"].hh_n_uni

165        1
176        1
213        1
228        1
240        1
          ..
5999878    1
5999941    1
5999956    1
5999969    1
6000020    1
Name: hh_n_uni, Length: 207412, dtype: int64

In [12]:
print(df_household.to_latex())

NameError: name 'df_household' is not defined

In [ ]:
df_household = df.groupby(["job_status"])[["hh_n_total", "hh_n_kindergarten", "hh_n_school", "hh_n_uni", "hh_n_working", "hh_n_none", ]].mean()
df_household = df_household.round(2)

job_status,None,kindergarten_kid,pupil,student,working
hh_n_total,2.27,3.85,4.09,2.47,2.46
hh_n_kindergarten,0.08,1.34,0.22,0.07,0.15
hh_n_school,0.24,0.60,1.88,0.24,0.40
hh_n_uni,0.02,0.03,0.03,1.10,0.04
hh_n_working,0.34,1.24,1.27,0.76,1.55
hh_n_none,1.59,0.64,0.69,0.30,0.31


In [ ]:
print(df_household.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &  hh\_n\_kindergarten &  hh\_n\_school &  hh\_n\_uni &  hh\_n\_working &  hh\_n\_none &  hh\_n\_total \\
job\_status       &                    &              &           &               &            &             \\
\midrule
None             &               0.08 &         0.24 &      0.02 &          0.34 &       1.59 &        2.27 \\
kindergarten\_kid &               1.34 &         0.60 &      0.03 &          1.24 &       0.64 &        3.85 \\
pupil            &               0.22 &         1.88 &      0.03 &          1.27 &       0.69 &        4.09 \\
student          &               0.07 &         0.24 &      1.10 &          0.76 &       0.30 &        2.47 \\
working          &               0.15 &         0.40 &      0.04 &          1.55 &       0.31 &        2.46 \\
\bottomrule
\end{tabular}



In [ ]:
df.groupby(["state", "student"])["hh_sum_work_hours"].mean()

state  student
2      0           6.099447
       1           8.174459
8      0           8.053618
       1          10.058744
9      0           7.554268
       1           8.974123
10     0           6.908731
       1           6.394527
Name: hh_sum_work_hours, dtype: float64

In [ ]:
df.groupby(["state", "school"])["hh_sum_school"].mean()

state  school
2      0         0.370787
       1         1.946818
8      0         0.346102
       1         1.997435
9      0         0.303203
       1         1.828299
10     0         0.313653
       1         1.713296
Name: hh_sum_school, dtype: float64